In [1]:
from officelib.xllib import *
import numpy as np
from pywintypes import com_error
import os
import re

In [101]:
_factors = {
    "hr": 24,
    "min": 1440,
    "sec": 86400,
    "day": 1,
}

def _alias(b, *a):
    for al in a:
        _factors[al] = _factors[b]
    
_alias("hr", "hour", "hours")
_alias("min", "minutes", "minute")
_alias("sec", "seconds", "second")
_alias("day", "days")
del _alias

def _getfactor(f):
    if isinstance(f, str):
        try:
            f = _factors[f]
        except KeyError:
            raise ValueError("Invalid time factor: '%s'"%f) from None
    return f
        

def group3(l):
    it = iter(l)
    yield from zip(it, it, it)

def floatify(l):    
    return tuple(map(float, l))

def formula_vars(formula):
    return re.findall(r"\{(.*?)\}", formula)

class ElapsedAlreadyExists(Exception):
    pass

class Analyzer():
    def __init__(self, wb_or_fn, xl=None):
        if xl is None:
            xl = Excel()
        self.xl = xl
            
        if isinstance(wb_or_fn, str):
            wb_or_fn = xl.Workbooks.Open(wb_or_fn)
        self.wb = wb_or_fn
        
        self.ws = self.wb.Worksheets(1)
        self.cells = self.ws.Cells
        self.cr = self.cells.Range
        
        self.ws2 = self.wb.Worksheets.Add()
        self.cells2 = self.ws2.Cells
        self.cr2 = self.cells2.Range
        
        self.interp_ref_vars = {
            "DOPV(%)",
            "DON2FlowActualRequest(%)",
            "DOO2FlowControllerRequestLimited(%)",
            "pHCO2ActualRequest(%)",
        }
        
    def kill(self):
        self.wb.Close(False)
        self.xl.Quit()
            
    def gdata(self, addr):
        cr = self.cr
        
        c1 = cr(addr)
        c2 = c1.End(xlc.xlDown).GetOffset(0, 1)
        data = cr(c1, c2).Value2
        return list(zip(*data))

    def find(self, var, thews=None):
        if thews is None:
            thews = self.ws
        cells = thews.Cells
        
        r = cells.Rows(1).EntireRow
        if r.Cells(1,1).Value2 == var:
            return r.Cells(1,1)
        v = r.Find(var)
        if v is None:
            raise ValueError(var)
        return v
    
    def elapsed(self, var, factor=24, ref=None):
        c, factor, ref = self.ck_elapsed(var, factor, ref)
        self.mk_elapsed(c, factor, ref)

    def ck_elapsed(self, var, factor, ref):
        factor = _getfactor(factor)
        c = self.find(var)
        if "Elapsed" in c.GetOffset(0,1).Value2:
            raise ElapsedAlreadyExists("Already has Elapsed Time")
        if ref is None:
            ref = c.GetOffset(1,0)
        return c, factor, ref

    def mk_elapsed(self, c, factor, ref):
        #c.GetOffset(0,1).EntireColumn.Insert()
        e1 = c.GetOffset(1,1)
        e2 = c.End(xlc.xlDown).GetOffset(0, 1)
        er = self.cr(e1,e2)

        formula = "=(%s-%s)*%d" % (c.GetOffset(1,0).GetAddress(0,0), ref.Address, factor)
        e1.Value = formula
        e1.NumberFormat = "0.00"
        e1.AutoFill(er)
        c.GetOffset(0,1).Value = "Elapsed(x%d)"%factor

    def vars(self):
        row1 = self.ws.UsedRange.Rows(1).Value2[0]
        ret = []
        for a,_,c in group3(row1):
            if a is None or c is not None:  # end of data
                break
            ret.append(a)
        return ret

    def allel(self):
        vs = vars()
        row2 = self.ws.UsedRange.Rows(2).Value2[0][:3*len(vs):3]
        m = row2[0]
        ii = 0
        for i, ts in enumerate(row2):
            if ts < m:
                m = ts
                ii = i

        mv = vs[ii]
        self.elapsed(mv, 24)
        ref = self.find(mv).GetOffset(1,0)
        for v in vs:
            if v == mv:
                continue
            self.elapsed(v, 24, ref)

    def allel2(self, vs):
        row2 = self.ws.UsedRange.Rows(2).Value2[0][:3*len(vs):3]
        m = row2[0]
        ii = 0
        for i, ts in enumerate(row2):
            if ts < m:
                m = ts
                ii = i
        mv = vs[ii]
        factor = 24
        
        for i, _ in enumerate(vs):
            self.ws.Columns(4*i + 2).Insert()
            
        ref = self.cells(2, 4*ii + 1)
        self.mk_elapsed(self.cells(1, 4*ii + 1), factor, ref)
        
        self.xl.Calculation = xlc.xlManual
        
        try:
            for i, v in enumerate(vs):
                if v == mv: 
                    continue
                c = self.cells(1, 4*i + 1)
                self.mk_elapsed(c, factor, ref)

        finally:
            self.xl.Calculation = xlc.xlAutomatic

    def xyd2(self, c):
        c2 = c.End(xlc.xlDown).GetOffset(0, 1)
        data = self.cr(c,c2).Value2
        return list(zip(*data))

    def i2xy(self, i):
        return self.xyd2(self.cells(2, 4*i + 2))
    
    def _mmc(self, i):
        c = self.cells(2, 4*i+2)
        min = c.Value2
        max = c.End(xlc.xlDown).Value2
        return min, max

    def allxd(self, vs):
        minx, maxx = self._mmc(0)
        
        xd = []
        for i,v in enumerate(vs):
            cminx, cmaxx = self._mmc(i)
            if cminx < minx: 
                minx = cminx
            if cmaxx > maxx:
                maxx = cmaxx
            if v in self.interp_ref_vars:
                colx, _ = self.i2xy(i)
                xd.append(colx)
                 
        space = np.linspace(minx, maxx, 4000)
        xd.append(space)
        return np.sort(np.concatenate(xd))

    def interpall(self, allx, vs):
        header = ["Elapsed"]
        data = [floatify(allx)]
        for i,v in enumerate(vs):
            vx, vy = self.i2xy(i)
            iy = np.interp(allx, vx, vy)
            header.append(v)
            data.append(floatify(iy))
        return header, data

    def interp2ws(self, header, data):
        h1 = self.cr2("A1"); h2 = h1.GetOffset(0, len(header) - 1)
        self.cr2(h1,h2).Value2 = [header]

        data = list(zip(*data))
        c1 = self.cr2("A2")
        c2 = c1.GetOffset(len(data)-1, len(data[0])-1)
        self.cr2(c1,c2).Value2 = data

    def graph(self, *vars):
        x1 = self.cr2("A2")
        x2 = x1.End(xlc.xlDown)
        xr = self.cr2(x1, x2)

        ch = CreateChart(self.ws2)
        PurgeSeriesCollection(ch)

        for v in vars:
            y0 = self.cells2.Find(v)
            if y0 is None:
                ch.Parent.Delete()
                raise ValueError(v)
            y2 = y0.End(xlc.xlDown)
            yr = self.cr2(y0.GetOffset(1,0), y2)
            s = CreateDataSeries(ch, xr, yr, v)
            s.MarkerSize = 3
        
        return ch

    def add_formula1(self, name, formula):
        fvars = formula_vars(formula)
        frefs = []
        for v in fvars:
            if v == "x":
                varc = self.cr2("A2")
            else:
                varc = self.find(v, self.ws2).GetOffset(1,0)
            frefs.append(varc)
        c = self.cr2("A1").End(xlc.xlToRight).GetOffset(0, 1)
        c.Value2 = name
        c1 = c.GetOffset(1,0)
        c2 = c.GetOffset(0,-1).End(xlc.xlDown).GetOffset(0,1)
        dest = self.cr2(c1, c2)
        for v,r in zip(fvars, frefs):
            formula = formula.replace("{%s}"%v, r.GetAddress(0,0))
        c1.Value = "="+formula
        c1.AutoFill(dest)

    def rename(self, col, new):
        c1 = self.find(col, self.ws2)
        c1.Value = new
        
    def analyze(self):
        with screen_lock(self.xl):
            vs = self.vars()
            print("    calculating elapsed time...")
            self.allel2(vs)
            print("    calculating interpolation points...")
            x = self.allxd(vs)
            print("    interpolating data...")
            h, d = self.interpall(x, vs)
            print("    pasting values into new worksheet...")
            self.interp2ws(h, d)
            
    def numfmt(self, col, fmt):
        c1 = self.find(col, self.ws2)
        c1.EntireColumn.NumberFormat = fmt
        
    def save(self, name):
        path = self.wb.Path
        if not name.endswith(".xlsx"):
            name += ".xlsx"
        newpath = os.path.join(path, name)
        self.save2(newpath)
        
    def save2(self, path):
        if os.path.exists(path) and os.path.isfile(path):
            os.remove(path)
        self.wb.SaveAs(path, FileFormat=xlc.xlOpenXMLWorkbook)
        
    def csvsave(self):
        name = self.wb.Name.replace(".csv", ".xlsx")
        self.save(name)

In [105]:
def run_script(a):
    with screen_lock(a.xl):
        a.analyze()

        print("    creating formula columns...")
        
        a.rename("DON2FlowActualRequest(%)", "N2 Flow (%)")
        a.rename("DOO2FlowControllerRequestLimited(%)", "O2 Flow (%)")
        a.rename("pHCO2ActualRequest(%)", "CO2 Flow (%)")
        a.rename("Elapsed", "Elapsed(hr)")

        a.add_formula1("Air Flow (%)", "100 - ({N2 Flow (%)} + {O2 Flow (%)} + {CO2 Flow (%)})")
        a.add_formula1("O2 headspace", "{N2 Flow (%)}*0.2095 + {O2 Flow (%)} + " + "{Air Flow (%)}")
        a.add_formula1("DO (equil, theoretical)", "{O2 headspace} * (100 / 20.95)")
        a.add_formula1("DO deficit", "{DO (equil, theoretical)} - {DOPV(%)}")

        print("    adding graphs...")
        
        ch = a.graph("O2 headspace", "DO (equil, theoretical)", "DO deficit")
        ch.Axes(xlc.xlValue, xlc.xlPrimary).MinimumScale = 0.00
        ch.Axes(xlc.xlCategory, xlc.xlPrimary).MaximumScale = 700

        ch = a.graph("Air Flow (%)", "N2 Flow (%)", "O2 Flow (%)")
        ch.Axes(xlc.xlValue, xlc.xlPrimary).MinimumScale = 0.00
        ch.Axes(xlc.xlCategory, xlc.xlPrimary).MaximumScale = 700

        to_fmt_000 = [
            "Air Flow (%)",
            "N2 Flow (%)",
            "CO2 Flow (%)",
            "O2 Flow (%)",
            "Elapsed(hr)",
            "DO (equil, theoretical)",
            "DO deficit",
            "DOPV(%)"
        ]

        print("    applying formatting...")
        
        for col in to_fmt_000:
            a.numfmt(col, "0.00")
            
        co1 = a.ws2.ChartObjects(1)
        co2 = a.ws2.ChartObjects(2)
        
        co1.Left = 20.0
        co1.Top = 50.0 
        co1.Width = 940.0 
        co1.Height = 180.0
        
        co2.Left = 20.0 
        co2.Top = 236.0 
        co2.Width = 940.0 
        co2.Height = 180.0

In [110]:
def runfolder(folder):
    files = [f for f in os.listdir(folder) if f.endswith(".csv")]
    xl = Excel()
    for f in files:
        runfile(xl, folder, f)
        
def runfile(xl, folder,f):
    try:
        fp = os.path.join(folder, f)
        print("analyzing %r..."%f)
        a = Analyzer(fp, xl)
        run_script(a)
    except Exception as e:
        print("    error occurred: " + str(e))
    finally:
        print("    saving...")
        a.csvsave()
        #a.wb.Close(False)
    print("    finished!")

In [109]:
runfile(pdd, "D763 Process Data.csv")

analyzing 'D763 Process Data.csv'...
    calculating elapsed time...
    calculating interpolation points...
    interpolating data...
    pasting values into new worksheet...
    creating formula columns...
    adding graphs...
    applying formatting...
    saving...
    finished!


In [107]:
pdd = 'C:\\Users\\Nathan\\Documents\\PBS\\Apps & Service\\Semma PD\\Process Data for Analysis'
runfolder(pdd)

analyzing 'D759 Process Data.csv'...
    calculating elapsed time...
    calculating interpolation points...
    interpolating data...
    pasting values into new worksheet...
    creating formula columns...
    adding graphs...
    applying formatting...
    saving...
    finished!
analyzing 'D760 Process Data.csv'...
    calculating elapsed time...
    calculating interpolation points...
    interpolating data...
    pasting values into new worksheet...
    creating formula columns...
    adding graphs...
    applying formatting...
    saving...
    finished!
analyzing 'D761 Process Data.csv'...
    calculating elapsed time...
    calculating interpolation points...
    interpolating data...
    pasting values into new worksheet...
    creating formula columns...
    adding graphs...
    applying formatting...
    saving...
    finished!
analyzing 'D762 Process Data.csv'...
    calculating elapsed time...
    calculating interpolation points...
    interpolating data...
    pasting v

In [120]:
xl.DisplayAlerts = False
for file in os.listdir(pdd):
    if not file.endswith(".xlsx"):
        continue
    print("fixing "+repr(file))
    fp = os.path.join(pdd, file)
    wb = xl.Workbooks.Open(fp)
    ws = wb.Worksheets(1)
    if ws.Name == "Sheet1":
        ws = wb.Worksheets(2)
    if ws.Name == "Sheet1":
        print("??? "+wb.Name)
        continue
    ws.Delete()
    wb.Save()

xl.DisplayAlerts = True
xl.Visible = True

fixing 'D759 Process Data.xlsx'
fixing 'D760 Process Data.xlsx'
fixing 'D761 Process Data.xlsx'
fixing 'D762 Process Data.xlsx'
fixing 'D763 Process Data.xlsx'
fixing 'D768 Process Data.xlsx'
fixing 'D789 Process Data.xlsx'
fixing 'D790 Process Data.xlsx'
fixing 'D792 Process Data.xlsx'


In [121]:
xl.Visible=True

In [124]:
xl.DisplayAlerts = False
xl.Visible=True
for file in os.listdir(pdd):
    if not file.endswith(".xlsx") or file.startswith("~"):
        continue
    print("fixing "+repr(file))
    fp = os.path.join(pdd, file)
    wb = xl.Workbooks.Open(fp)
    ws = wb.Worksheets(1)
    
    ch = ws.ChartObjects(2).Chart
    ch.Axes(xlc.xlValue, xlc.xlPrimary).MinimumScale = 0
    ch.Axes(xlc.xlCategory, xlc.xlPrimary).MaximumScale = 700
    
    ch = ws.ChartObjects(1).Chart
    ch.Axes(xlc.xlCategory, xlc.xlPrimary).MaximumScale = 700
    
    wb.Save()

xl.DisplayAlerts = True
xl.Visible = True

fixing 'D759 Process Data.xlsx'
fixing 'D760 Process Data.xlsx'
fixing 'D761 Process Data.xlsx'
fixing 'D762 Process Data.xlsx'
fixing 'D763 Process Data.xlsx'
fixing 'D768 Process Data.xlsx'
fixing 'D789 Process Data.xlsx'
fixing 'D790 Process Data.xlsx'
fixing 'D792 Process Data.xlsx'
